In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
import pandas as pd
import warnings 
warnings.filterwarnings(action='ignore')
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import re

In [ ]:
#경로 설정
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/모두의딥러닝')

In [ ]:
#파일 불러오기
train = pd.read_csv('train.csv', encoding = 'utf-8')
test = pd.read_csv('test_x.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('sample_submission.csv', encoding = 'utf-8')

In [ ]:
#train 데이터 살펴보기
train

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3
...,...,...,...
54874,54874,"“Is that you, Mr. Smith?” odin whispered. “I h...",2
54875,54875,"I told my plan to the captain, and between us ...",4
54876,54876,"""Your sincere well-wisher, friend, and sister...",1
54877,54877,“Then you wanted me to lend you money?”,3


In [ ]:
#test 데이터 살펴보기
test

,index,text
0,0,“Not at all. I think she is one of the most ch...
1,1,"""No,"" replied he, with sudden consciousness, ""..."
2,2,As the lady had stated her intention of scream...
3,3,“And then suddenly in the silence I heard a so...
4,4,His conviction remained unchanged. So far as I...
...,...,...
19612,19612,"At the end of another day or two, odin growing..."
19613,19613,"All afternoon we sat together, mostly in silen..."
19614,19614,"odin, having carried his thanks to odin, proc..."
19615,19615,"Soon after this, upon odin's leaving the room,..."


In [ ]:
#sample_submission
sample_submission

,index,0,1,2,3,4
0,0,0,0,0,0,0
1,1,0,0,0,0,0
2,2,0,0,0,0,0
3,3,0,0,0,0,0
4,4,0,0,0,0,0
...,...,...,...,...,...,...
19612,19612,0,0,0,0,0
19613,19613,0,0,0,0,0
19614,19614,0,0,0,0,0
19615,19615,0,0,0,0,0


# **전처리**

In [ ]:
#부호를 제거해주는 함수
def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)

train['text']=train['text'].apply(alpha_num)

In [ ]:
#부호가 사라진 것을 확인할 수 있습니다.
train

,index,text,author
0,0,He was almost choking There was so much so muc...,3
1,1,Your sister asked for it I suppose,2
2,2,She was engaged one day as she walked in peru...,1
3,3,The captain was in the porch keeping himself c...,4
4,4,Have mercy gentlemen odin flung up his hands D...,3
...,...,...,...
54874,54874,Is that you Mr Smith odin whispered I hardly d...,2
54875,54875,I told my plan to the captain and between us w...,4
54876,54876,Your sincere wellwisher friend and sister LUC...,1
54877,54877,Then you wanted me to lend you money,3


In [ ]:
# 불용어 제거해주는 함수
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)

# 불용어
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

In [ ]:
#전처리 적용
train['text'] = train['text'].str.lower()
test['text'] = test['text'].str.lower()
train['text'] = train['text'].apply(alpha_num).apply(remove_stopwords)
test['text'] = test['text'].apply(alpha_num).apply(remove_stopwords)

In [ ]:
# train test 분리
X_train = np.array([x for x in train['text']])
X_test = np.array([x for x in test['text']])
y_train = np.array([x for x in train['author']])

In [ ]:
X_train

array(['almost choking much much wanted say strange exclamations came lips pole gazed fixedly bundle notes hand looked odin evident perplexity',
       'sister asked suppose',
       'engaged one day walked perusing janes last letter dwelling passages proved jane not written spirits instead surprised mr odin saw looking odin meeting putting away letter immediately forcing smile said',
       ..., 'sincere wellwisher friend sister lucy odin',
       'wanted lend money', 'certainly not occurred said yes like'],
      dtype='<U1433')

# **모델링**

In [ ]:
#파라미터 설정
vocab_size = 20000
embedding_dim = 16
max_length = 500
padding_type='post'
#oov_tok = "<OOV>"

In [ ]:
#tokenizer에 fit
tokenizer = Tokenizer(num_words = vocab_size)#, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

In [ ]:
#데이터를 sequence로 변환해주고 padding 해줍니다.
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)

test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)

In [ ]:
#가벼운 NLP모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [ ]:
# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 16)           320000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 24)                408       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 125       
Total params: 320,533
Trainable params: 320,533
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# fit model
num_epochs = 20
history = model.fit(train_padded, y_train, 
                    epochs=num_epochs, verbose=2, 
                    validation_split=0.2)

Epoch 1/20
1372/1372 - 8s - loss: 1.5671 - accuracy: 0.2773 - val_loss: 1.5548 - val_accuracy: 0.2697
Epoch 2/20
1372/1372 - 8s - loss: 1.4601 - accuracy: 0.3869 - val_loss: 1.3426 - val_accuracy: 0.4468
Epoch 3/20
1372/1372 - 7s - loss: 1.2432 - accuracy: 0.4767 - val_loss: 1.2220 - val_accuracy: 0.4704
Epoch 4/20
1372/1372 - 8s - loss: 1.1403 - accuracy: 0.5100 - val_loss: 1.1419 - val_accuracy: 0.5146
Epoch 5/20
1372/1372 - 8s - loss: 1.0755 - accuracy: 0.5397 - val_loss: 1.1052 - val_accuracy: 0.5407
Epoch 6/20
1372/1372 - 7s - loss: 1.0242 - accuracy: 0.5658 - val_loss: 1.0662 - val_accuracy: 0.5604
Epoch 7/20
1372/1372 - 7s - loss: 0.9716 - accuracy: 0.5948 - val_loss: 1.0334 - val_accuracy: 0.5814
Epoch 8/20
1372/1372 - 7s - loss: 0.9181 - accuracy: 0.6277 - val_loss: 1.0064 - val_accuracy: 0.5909
Epoch 9/20
1372/1372 - 7s - loss: 0.8579 - accuracy: 0.6614 - val_loss: 0.9545 - val_accuracy: 0.6318
Epoch 10/20
1372/1372 - 7s - loss: 0.7985 - accuracy: 0.6938 - val_loss: 0.9149 - 

In [ ]:
# predict values
pred = model.predict_proba(test_padded)

Instructions for updating:
Please use `model.predict()` instead.


In [ ]:
pred

array([[7.2716934e-06, 9.9575341e-01, 3.8434935e-03, 3.0491300e-04,
        9.0938614e-05],
       [1.5512076e-01, 5.7651877e-01, 3.7415259e-02, 1.2747859e-02,
        2.1819726e-01],
       [9.9935681e-01, 2.4487033e-06, 5.5710226e-04, 9.8968862e-08,
        8.3554820e-05],
       ...,
       [1.2068418e-05, 9.9978465e-01, 9.9302397e-06, 3.3239687e-06,
        1.9010104e-04],
       [4.0114606e-05, 9.9942762e-01, 8.0032391e-05, 1.5229148e-06,
        4.5057075e-04],
       [9.9935144e-01, 2.7795878e-07, 5.1516254e-04, 3.6457701e-07,
        1.3282150e-04]], dtype=float32)

In [ ]:
# submission
sample_submission[['0','1','2','3','4']] = pred
sample_submission

,index,0,1,2,3,4
0,0,7.271693e-06,9.957534e-01,3.843494e-03,3.049130e-04,0.000091
1,1,1.551208e-01,5.765188e-01,3.741526e-02,1.274786e-02,0.218197
2,2,9.993568e-01,2.448703e-06,5.571023e-04,9.896886e-08,0.000084
3,3,2.230293e-04,8.340119e-07,9.908852e-01,8.676366e-12,0.008891
4,4,9.796013e-01,5.727318e-05,1.152378e-03,1.883311e-02,0.000356
...,...,...,...,...,...,...
19612,19612,2.474993e-07,9.999957e-01,2.279798e-11,4.052209e-09,0.000004
19613,19613,3.047024e-05,2.583957e-07,1.990941e-05,8.589753e-13,0.999949
19614,19614,1.206842e-05,9.997846e-01,9.930240e-06,3.323969e-06,0.000190
19615,19615,4.011461e-05,9.994276e-01,8.003239e-05,1.522915e-06,0.000451


In [ ]:
sample_submission.to_csv('submission.csv', index = False, encoding = 'utf-8')